In [2]:
import numpy as np
import pandas as pd

In [3]:
def process_data_with_hourly_index(data: pd.DataFrame) -> pd.DataFrame:
    """
    Rebuilds a clean Datetime column assuming hourly frequency,
    starting from the first known correct timestamp.
    """

    data = data.copy()

    # Find columns containing "Date"
    date_cols = [col for col in data.columns if "Date" in col]

    # Rename them to "Datetime"
    for col in date_cols:
        data = data.rename(columns={col: "Datetime"})

    # Convert to datetime, coercing errors
    data["Datetime"] = pd.to_datetime(
        data["Datetime"], errors="coerce", dayfirst=True)

    # If dates are in descending order, reverse the DataFrame
    if data.Datetime.iloc[0] > data.Datetime.iloc[-1]:
        data = data.iloc[::-1]
        data = data.reset_index(drop=True)

    # Identify duplicate timestamps
    duplicates = data['Datetime'].value_counts()
    duplicates = duplicates[duplicates == 2].index

    # For those, drop duplicates but keep the first one
    mask = data['Datetime'].isin(duplicates)
    data = pd.concat([
        data[~mask],                           # keep all non-duplicates
        data[mask].drop_duplicates('Datetime')     # keep one of each duplicate pair
    ]).reset_index(drop=True)

    # First known correct timestamp
    start = pd.Timestamp("2017-08-17 04:00:00")

    # Build a new datetime range: one row per hour
    data["Datetime"] = pd.date_range(start=start, periods=len(data), freq="h")

    return data

In [4]:
data = pd.read_csv('Binance_BTCUSDT_1h.csv')
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.758580e+12,22/09/25 23:00,BTCUSDT,112643.25,112739.14,112592.20,112650.99,135.310950,1.524494e+07,34083
1,1.758580e+12,22/09/25 22:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.607150,3.264691e+07,42836
2,1.758570e+12,22/09/25 21:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931
3,1.758570e+12,22/09/25 20:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
4,1.758570e+12,22/09/25 19:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
...,...,...,...,...,...,...,...,...,...,...
70822,1.502960e+12,17/08/17 8:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
70823,1.502950e+12,17/08/17 7:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
70824,1.502950e+12,17/08/17 6:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
70825,1.502950e+12,17/08/17 5:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102


In [5]:
data['Date'].value_counts()

Date
00:00.0           597
28:14.8            43
02/03/25 16:00      2
05/03/25 23:00      2
05/03/25 11:00      2
                 ... 
22/12/22 7:00       1
22/12/22 6:00       1
22/12/22 5:00       1
22/12/22 4:00       1
17/08/17 4:00       1
Name: count, Length: 69683, dtype: int64

In [6]:
data = process_data_with_hourly_index(data)
data

/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_16938/3951306850.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Datetime"] = pd.to_datetime(


,Unix,Datetime,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.502940e+12,2017-08-17 04:00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,2.023661e+05,171
1,1.502950e+12,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102
2,1.502950e+12,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
3,1.502950e+12,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
4,1.502960e+12,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
...,...,...,...,...,...,...,...,...,...,...
70316,1.741720e+12,2025-08-25 00:00:00,BTCUSDT,83280.95,83617.40,82846.31,83104.01,1549.392610,1.289456e+08,255838
70317,1.741720e+12,2025-08-25 01:00:00,BTCUSDT,83104.01,83170.38,82586.84,82799.68,845.304400,7.005551e+07,148383
70318,1.741730e+12,2025-08-25 02:00:00,BTCUSDT,82799.68,83265.38,82799.67,83110.72,576.901240,4.794675e+07,100776
70319,1.741730e+12,2025-08-25 03:00:00,BTCUSDT,83110.71,83306.87,82981.30,83133.48,560.569900,4.660952e+07,117908


In [7]:
data['Datetime'].value_counts()

Datetime
2017-08-17 04:00:00    1
2022-12-22 10:00:00    1
2022-12-22 16:00:00    1
2022-12-22 15:00:00    1
2022-12-22 14:00:00    1
                      ..
2020-04-20 03:00:00    1
2020-04-20 04:00:00    1
2020-04-20 05:00:00    1
2020-04-20 06:00:00    1
2025-08-25 04:00:00    1
Name: count, Length: 70321, dtype: int64

In [63]:
def process_data_with_hourly_index(data: pd.DataFrame) -> pd.DataFrame:
    """
    Rebuilds a clean Datetime column assuming hourly frequency,
    starting from the first known correct timestamp.
    """

    data = data.copy()

    # Find columns containing "Date"
    date_cols = [col for col in data.columns if "Date" in col]

    # Rename them to "Datetime"
    for col in date_cols:
        data = data.rename(columns={col: "Datetime"})
    
    # Convert to datetime, coercing errors
    data["Datetime"] = pd.to_datetime(
        data["Datetime"], errors="coerce", dayfirst=True)

    # If dates are in descending order, reverse the DataFrame
    if data.Datetime.iloc[0] > data.Datetime.iloc[-1]:
        data = data.iloc[::-1]
        data = data.reset_index(drop=True)
    
    # Identify duplicate timestamps
    duplicates = data['Datetime'].value_counts()
    duplicates = duplicates[duplicates == 2].index

    # For those, drop duplicates but keep the first one
    mask = data['Datetime'].isin(duplicates)
    data = pd.concat([
        data[~mask],                           # keep all non-duplicates
        data[mask].drop_duplicates('Datetime')     # keep one of each duplicate pair
    ]).reset_index(drop=True)
    
    # First known correct timestamp
    #start = pd.Timestamp("2017-08-17 04:00:00")

    # Build a new datetime range: one row per hour
    #data["Datetime"] = pd.date_range(start=start, periods=len(data), freq="h")

    return data

In [70]:
data = pd.read_csv('Binance_BTCUSDT_1h.csv')
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.758580e+12,22/09/25 23:00,BTCUSDT,112643.25,112739.14,112592.20,112650.99,135.310950,1.524494e+07,34083
1,1.758580e+12,22/09/25 22:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.607150,3.264691e+07,42836
2,1.758570e+12,22/09/25 21:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931
3,1.758570e+12,22/09/25 20:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
4,1.758570e+12,22/09/25 19:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
...,...,...,...,...,...,...,...,...,...,...
70822,1.502960e+12,17/08/17 8:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
70823,1.502950e+12,17/08/17 7:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
70824,1.502950e+12,17/08/17 6:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
70825,1.502950e+12,17/08/17 5:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102


In [37]:
data['Date'].value_counts()

Date
00:00.0           597
28:14.8            43
02/03/25 16:00      2
05/03/25 23:00      2
05/03/25 11:00      2
                 ... 
22/12/22 7:00       1
22/12/22 6:00       1
22/12/22 5:00       1
22/12/22 4:00       1
17/08/17 4:00       1
Name: count, Length: 69683, dtype: int64

In [38]:
data = process_data_with_hourly_index(data)
data

/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_16938/1061357490.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Datetime"] = pd.to_datetime(


,Unix,Datetime,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.739322e+15,2017-08-17 04:00:00,BTCUSDT,96060.00,96263.99,95817.69,96207.32,373.26635,3.583996e+07,94740
1,1.739326e+15,2017-08-17 05:00:00,BTCUSDT,96207.33,96214.70,95384.61,95579.14,657.69595,6.300092e+07,111288
2,1.739329e+15,2017-08-17 06:00:00,BTCUSDT,95579.13,95762.67,95100.00,95299.27,799.73342,7.634759e+07,147072
3,1.739333e+15,2017-08-17 07:00:00,BTCUSDT,95299.28,95611.12,95267.71,95556.98,551.85072,5.267914e+07,135940
4,1.739336e+15,2017-08-17 08:00:00,BTCUSDT,95556.99,95933.30,95514.60,95886.54,318.13104,3.046700e+07,86285
...,...,...,...,...,...,...,...,...,...,...
70316,1.758570e+12,2025-08-25 00:00:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.37365,1.467768e+08,126232
70317,1.758570e+12,2025-08-25 01:00:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.84005,6.707508e+07,93553
70318,1.758570e+12,2025-08-25 02:00:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.31156,3.307493e+07,42931
70319,1.758580e+12,2025-08-25 03:00:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.60715,3.264691e+07,42836


In [39]:
data['Datetime'].value_counts()

Datetime
2017-08-17 04:00:00    1
2022-12-22 10:00:00    1
2022-12-22 16:00:00    1
2022-12-22 15:00:00    1
2022-12-22 14:00:00    1
                      ..
2020-04-20 03:00:00    1
2020-04-20 04:00:00    1
2020-04-20 05:00:00    1
2020-04-20 06:00:00    1
2025-08-25 04:00:00    1
Name: count, Length: 70321, dtype: int64

In [64]:
data = pd.read_csv('Binance_BTCUSDT_1h.csv')
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.758580e+12,22/09/25 23:00,BTCUSDT,112643.25,112739.14,112592.20,112650.99,135.310950,1.524494e+07,34083
1,1.758580e+12,22/09/25 22:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.607150,3.264691e+07,42836
2,1.758570e+12,22/09/25 21:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931
3,1.758570e+12,22/09/25 20:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
4,1.758570e+12,22/09/25 19:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
...,...,...,...,...,...,...,...,...,...,...
70822,1.502960e+12,17/08/17 8:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
70823,1.502950e+12,17/08/17 7:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
70824,1.502950e+12,17/08/17 6:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
70825,1.502950e+12,17/08/17 5:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102


In [55]:
duplicates = data['Date'].value_counts()
duplicates = duplicates[duplicates == 2].index
duplicates.value_counts()

Date
02/03/25 16:00    1
12/02/25 10:00    1
11/03/25 11:00    1
11/03/25 10:00    1
11/03/25 9:00     1
                 ..
02/03/25 2:00     1
02/03/25 3:00     1
02/03/25 4:00     1
02/03/25 5:00     1
18/02/25 1:00     1
Name: count, Length: 506, dtype: int64

In [27]:
data['Date'].value_counts()

Date
00:00.0           597
28:14.8            43
02/03/25 16:00      2
05/03/25 23:00      2
05/03/25 11:00      2
                 ... 
22/12/22 7:00       1
22/12/22 6:00       1
22/12/22 5:00       1
22/12/22 4:00       1
17/08/17 4:00       1
Name: count, Length: 69683, dtype: int64

In [30]:
data[data['Date'] == '00:00.0']

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
66167,1.519860e+12,00:00.0,BTCUSDT,10430.03,10469.69,10300.00,10326.76,1863.280232,1.928585e+07,13963
66168,1.519860e+12,00:00.0,BTCUSDT,10520.55,10574.63,10429.11,10430.03,779.519688,8.188007e+06,8861
66169,1.519850e+12,00:00.0,BTCUSDT,10670.00,10722.00,10520.53,10520.53,1147.463424,1.221455e+07,10607
66170,1.519850e+12,00:00.0,BTCUSDT,10563.79,10691.32,10537.64,10670.00,993.480740,1.056937e+07,11624
66171,1.519840e+12,00:00.0,BTCUSDT,10480.88,10570.89,10474.79,10560.02,500.904197,5.266215e+06,7019
...,...,...,...,...,...,...,...,...,...,...
66802,1.517460e+12,00:00.0,BTCUSDT,10069.77,10256.00,10000.01,10245.79,649.939854,6.582218e+06,7431
66803,1.517450e+12,00:00.0,BTCUSDT,10199.61,10250.79,9959.04,10069.80,739.435309,7.459190e+06,9014
66804,1.517450e+12,00:00.0,BTCUSDT,10249.43,10317.73,10176.89,10199.61,479.412562,4.925488e+06,6550
66805,1.517450e+12,00:00.0,BTCUSDT,10263.18,10328.98,10216.00,10247.49,591.515725,6.075266e+06,7011


In [31]:
data[data['Date'] == '02/03/25 16:00']

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
4553,1.740931e+15,02/03/25 16:00,BTCUSDT,87428.0,91959.99,87278.51,91200.0,11741.72701,1.056043e+09,1273816
4554,1.740930e+12,02/03/25 16:00,BTCUSDT,87428.0,91959.99,87278.51,91200.0,11741.72701,1.056043e+09,1273816


In [71]:
data["Date"] = pd.to_datetime(data["Date"], errors="coerce", dayfirst=True)

/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_16938/967486765.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], errors="coerce", dayfirst=True)


In [72]:
# Elimina duplicados basados en la columna Date o en OHLCV completos
df = data.drop_duplicates(subset=["Open","High","Low","Close","Volume BTC","Volume USDT","tradecount"]).reset_index(drop=True)
df

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.758580e+12,2025-09-22 23:00:00,BTCUSDT,112643.25,112739.14,112592.20,112650.99,135.310950,1.524494e+07,34083
1,1.758580e+12,2025-09-22 22:00:00,BTCUSDT,112969.99,112970.00,112594.33,112643.25,289.607150,3.264691e+07,42836
2,1.758570e+12,2025-09-22 21:00:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931
3,1.758570e+12,2025-09-22 20:00:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
4,1.758570e+12,2025-09-22 19:00:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
...,...,...,...,...,...,...,...,...,...,...
70316,1.502960e+12,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
70317,1.502950e+12,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
70318,1.502950e+12,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
70319,1.502950e+12,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102


In [73]:
df['Date'].value_counts()

Date
2025-10-03 00:00:00    597
2025-09-22 23:00:00      1
2020-05-12 02:00:00      1
2020-05-11 20:00:00      1
2020-05-11 21:00:00      1
                      ... 
2023-01-05 09:00:00      1
2023-01-05 08:00:00      1
2023-01-05 07:00:00      1
2023-01-05 06:00:00      1
2017-08-17 04:00:00      1
Name: count, Length: 69682, dtype: int64

In [65]:
data = process_data_with_hourly_index(data)
data

/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_16938/580077910.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Datetime"] = pd.to_datetime(


,Unix,Datetime,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1.502940e+12,2017-08-17 04:00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,2.023661e+05,171
1,1.502950e+12,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102
2,1.502950e+12,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
3,1.502950e+12,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
4,1.502960e+12,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
...,...,...,...,...,...,...,...,...,...,...
70316,1.741720e+12,2025-03-11 19:00:00,BTCUSDT,83280.95,83617.40,82846.31,83104.01,1549.392610,1.289456e+08,255838
70317,1.741720e+12,2025-03-11 20:00:00,BTCUSDT,83104.01,83170.38,82586.84,82799.68,845.304400,7.005551e+07,148383
70318,1.741730e+12,2025-03-11 21:00:00,BTCUSDT,82799.68,83265.38,82799.67,83110.72,576.901240,4.794675e+07,100776
70319,1.741730e+12,2025-03-11 22:00:00,BTCUSDT,83110.71,83306.87,82981.30,83133.48,560.569900,4.660952e+07,117908


In [66]:
data['Datetime'].value_counts()

Datetime
2025-10-03 00:00:00    597
2017-08-17 04:00:00      1
2023-01-05 13:00:00      1
2023-01-05 19:00:00      1
2023-01-05 18:00:00      1
                      ... 
2020-05-12 06:00:00      1
2020-05-12 07:00:00      1
2020-05-12 08:00:00      1
2020-05-12 09:00:00      1
2025-03-11 23:00:00      1
Name: count, Length: 69682, dtype: int64

In [74]:
# Identify duplicate timestamps
duplicates = data['Datetime'].value_counts()
duplicates = duplicates[duplicates == 2].index

# For those, drop duplicates but keep the first one
mask = data['Datetime'].isin(duplicates)
data = pd.concat([
    data[~mask],                           # keep all non-duplicates
    data[mask].drop_duplicates('Datetime')     # keep one of each duplicate pair
]).reset_index(drop=True)

KeyError: 'Datetime'